In [1]:
import seaborn as sns
from igraph import Graph as iGraph
from py2neo import Graph as pGraph
import pandas as pd
import dgl
import torch

Using backend: pytorch


链接图数据库，导入csv文件，分别是  
node_gzh.csv 包含节点属性  
node_rela_gzh.csv 包含节点间关系

In [ ]:
g = pGraph('http://localhost:7474',auth=("neo4j", "123456"))

In [ ]:
g.run('CREATE INDEX ON :gzh(item)')

In [ ]:
g.run('using periodic commit 10000 load csv with headers from "file:/node_gzh.csv" \
as line with line create (:gzh {item:line.item,  trans_amount_sum:line.trans_amount_sum,\
trans_cnt:line.trans_cnt,  type:line.type});')

In [ ]:
g.run('using periodic commit 10000 load csv with headers from "file:/node_rela_gzh.csv" as line \
match (from:gzh {item: line.item_l}),(to:gzh {item:line.item_r}) merge (from)-[c:gzh{relation:line.relation}]->(to)')

首先查询所有需要的节点，然后要统计出节点数量，这个很关键。在建图时需要用到

In [322]:
query = """
MATCH (n) RETURN n.item
"""

In [327]:
ndata  = g.run(query)
number_of_nodes=len(ndata.to_data_frame().astype('int'))
number_of_nodes

26077

读入边关系，边属性和节点属性,需要注意，你用来当做节点索引的属性必须是从0开始的，否则会导致数据无法对齐，会默认创造一个0节点  
但是我在这个数据正好是从1开始，需要处理一下

In [278]:
all_query = """
MATCH (p1:gzh)-[e:gzh]->(p2:gzh) 
return (p1.item) as src,(p2.item) as dst,e.relation as relation
"""

没想到什么好办法，直接减个1吧。。这样就是从0开始了。  
ps:后面还有个坑，因为这个是按照边关系建图，恰好最后一个节点他没有边关系，  
所以在导入节点特征的时候汇报错，在这里要按照节点的真实数量预设好num_nodes = number_of_nodes

In [329]:
data  = g.run(all_query)
graphdf=data.to_data_frame()
src,dst =  (graphdf[graphdf.columns[0]].astype('int').to_numpy()-1),(graphdf[graphdf.columns[1]].astype('int').to_numpy()-1)
test=dgl.graph((src, dst),num_nodes=number_of_nodes)

In [378]:
rquery = graphdf[['src','dst']].astype('int')-1
rquery.max()

src    26067
dst    26075
dtype: int32

In [331]:
test

Graph(num_nodes=26077, num_edges=212371,
      ndata_schemes={}
      edata_schemes={})

In [332]:
test.nodes()

tensor([    0,     1,     2,  ..., 26074, 26075, 26076])

可以设置边的属性，作为权重，但必须要转化为tensor的形式

In [333]:
test.edata['weight']  = torch.rand([len(src),1]).double()

这意味着如果是字符串的类别变量必须进行编码

In [334]:
graphdf['relation'].value_counts()

交易卡号    152903
设备关联     39881
身份证号     19587
Name: relation, dtype: int64

In [335]:
codes,keys=pd.factorize(graphdf['relation'])
relation_dict = {key: values for (key, values) in zip(keys, np.unique(codes))}
graphdf['label'] = codes
test.edata['label']  = torch.tensor(graphdf['label'])

In [336]:
relation_dict

{'设备关联': 0, '交易卡号': 1, '身份证号': 2}

可以查看某一边属性

In [337]:
test.edata['label']

tensor([0, 0, 0,  ..., 2, 2, 2])

可以对边属性进行筛选

In [299]:
def edges_with_feature_more_than_x(edges):
    return (edges.data['weight'] < 0.5).squeeze(1)

In [300]:
def edges_with_feature_one(edges):
    return (edges.data['label'] == 1)

In [338]:
test.filter_edges(edges_with_feature_more_than_x)

tensor([     1,      2,      4,  ..., 212366, 212368, 212370])

In [339]:
test

Graph(num_nodes=26077, num_edges=212371,
      ndata_schemes={}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.int64)})

在完成边的建立和属性设置后，可以把节点的属性导入

In [340]:
node_query = """
MATCH (p1:gzh)
return p1.item as id,p1.trans_amount_sum as trans_amount_sum,p1.trans_cnt as trans_cnt
"""

In [343]:
ndata = g.run(node_query)
nodedf = ndata.to_data_frame().astype('float').sort_values(by='id').reset_index(drop=True)
nodedf['id'] = nodedf['id']-1
trans_amount_sum = nodedf['trans_amount_sum'].to_numpy()
test.ndata['trans_amount_sum']  = torch.tensor(trans_amount_sum)

可以对节点属性进行筛选

In [345]:
def nodes_with_feature_one(edges):
    return (edges.data['trans_amount_sum'] <1000)

可以通过节点列表来构建子图

In [346]:
subtest = test.subgraph(test.filter_nodes(nodes_with_feature_one))
subtest

Graph(num_nodes=5674, num_edges=196738,
      ndata_schemes={'trans_amount_sum': Scheme(shape=(), dtype=torch.float64), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [29]:
def neo4j_to_dgl(graphdata):
    graphdf=graphdata.to_data_frame()
    columns1,columns2= graphdf.columns[0],graphdf.columns[1]
    columns3 = graphdf.columns[2]
    src,dst = graphdf[columns1].astype('int').to_numpy(),graphdf[columns2].astype('int').to_numpy()
    relation = graphdf[columns3].astype('float').to_numpy()
    g=dgl.graph((src, dst))
    g.edata[columns3]  = torch.tensor(relation)
    return g

In [347]:
test

Graph(num_nodes=26077, num_edges=212371,
      ndata_schemes={'trans_amount_sum': Scheme(shape=(), dtype=torch.float64)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float64), 'label': Scheme(shape=(), dtype=torch.int64)})

## 提取聚合特征

dgl内置消息传递和聚合函数  
https://docs.dgl.ai/api/python/dgl.function.html#message-functions

在DGL中，消息函数 接受一个参数 edges，这是一个 EdgeBatch 的实例,  
在消息传递时，它被DGL在内部生成以表示一批边。  
edges 有 src、 dst 和 data 共3个成员属性， 分别用于访问源节点、目标节点和边的特征

可以使用内部的函数也可以自定义，注意下面两个函数的效果是一样的,都是把source的信息乘以边的属性，再传递到destination

In [348]:
def udf_u_mul_e(edges):
    return {'m' : edges.src['trans_amount_sum'] * edges.data['weight']}

In [349]:
dgl.function.u_mul_e('trans_amount_sum', 'weight', 'm')

还可把source的信息和边上的信息直接拼接。

In [350]:
def u_cat_e(edges):
    return {'m': torch.hstack([edges.src['trans_amount_sum'],edges.data['trans_amount_sum']])}  

DGL支持内置的聚合函数 sum、 max、 min 和 mean 操作。 聚合函数通常有两个参数，  
它们的类型都是字符串。一个用于指定 mailbox 中的字段名，一个用于指示目标节点特征的字段名，  
nodes.mailbox['m']是之前在消息传递函数里，该节点从其他节点收集到的聚合信息，可以对他进行求和，求均值等操作。    
这个m是自己设置的，取决于之前在消息传递的时候把这些暂存的聚合信息命名为什么。

对接收到消息求和的用户定义函数 sum的两种写法，直接内置的函数比自定义的效率高。

In [351]:
def reduce_func(nodes):
     return {'h': torch.sum(nodes.mailbox['m'], dim=1)}

In [352]:
dgl.function.sum('m','agg_trans_amount_sum')

在DGL中，也可以在不涉及消息传递的情况下，通过 apply_edges() 单独调用逐边计算。  
apply_edges() 的参数是一个消息函数。并且在默认情况下，这个接口将更新所有的边。例如：

In [353]:
test.apply_edges(dgl.function.copy_u('trans_amount_sum', 'm'))

可以发现边属性多了一个m  
个人理解m就是个公路，apply_edges就是把货物从出发点发出，在公路上运输的阶段

举个例子，m的第一个值61769 对应29到4101这条关系，  
代表着从29号节点发出了61769个货物给4101号节点，这些货物还没到4101节点，暂存在公路m上  
可以发现29的trans_amount_sum 确实是61769

In [361]:
test.edata['m']

tensor([61769.0000, 87632.5000, 26290.0000,  ..., 53494.0000, 43699.0000,
        43699.0000], dtype=torch.float64)

In [354]:
test.edges()

(tensor([   29,    32,    34,  ..., 26065, 26067, 26067]),
 tensor([ 4101,   102,  1393,  ..., 25068, 26069, 26071]))

In [358]:
nodedf[29:30]

,id,trans_amount_sum,trans_cnt
29,29.0,61769.0,7.0


对于消息传递， update_all() 是一个高级API。它在单个API调用里合并了消息生成、 消息聚合和节点特征更新，  
这为从整体上进行系统优化提供了空间。

直接用apply edges很容易占用大量内存，
因为作为计算的中间变量没必要保存下来，
最好是计算完毕得到结果之后直接存储到目标节点的属性中，
这样时间和空间上的效率都高的多。

update_all() 的参数是一个消息函数、一个聚合函数和一个更新函数。   
更新函数是一个可选择的参数，用户也可以不使用它，而是在 update_all 执行完后直接对节点特征进行操作。  
由于更新函数通常可以用纯张量操作实现，所以DGL不推荐在 update_all 中指定更新函数。例如：

In [362]:
def update_all_example(graph):
    # 在graph.ndata['ft']中存储结果
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'),
                     fn.sum('m', 'ft'))
    # 在update_all外调用更新函数会带来大量额外计算
    #相当于 final_fti=2∗∑j∈N(i)(ftj∗aij)
    final_ft = graph.ndata['ft'] * 2
    return final_ft

一般来说对一度联系人进行聚合的话只需要

In [363]:
#未加权
test.update_all(dgl.function.copy_u('trans_amount_sum', 'm'),dgl.function.sum('m','aggsum_trans_amount_sum'))

In [364]:
#加权
test.update_all(dgl.function.u_mul_e('trans_amount_sum', 'weight', 'm'),dgl.function.sum('m','weighted_aggsum_trans_amount_sum'))

检查一下是否正确  
可以看到1号节点的值为15  

In [375]:
test.ndata['aggsum_trans_amount_sum'][:10]

tensor([0.0000e+00, 1.5000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        9.0145e+04, 0.0000e+00, 0.0000e+00, 2.4481e+05], dtype=torch.float64)

查看有哪些节点的目的地是1号节点，发现只有一个44号节点。对应的是关系表里的14号关系

In [381]:
rquery[rquery['dst']==1]

,src,dst
14,44,1


44号节点的值确实是15，聚合计算结果与实际情况一致。

In [382]:
nodedf[nodedf['id']==44]

,id,trans_amount_sum,trans_cnt
44,44.0,15.0,1.0


接下来检查加权聚合结果  
可以看到1号节点加权聚合结果为4.989 

In [384]:
test.ndata['weighted_aggsum_trans_amount_sum'][1]

tensor([4.9893], dtype=torch.float64)

由于他们对应的是第14号关系，故查的权重为0.3326

In [386]:
test.edata['weight'][14]

tensor([0.3326], dtype=torch.float64)

这就是一度联系人加权特征的计算过程

In [387]:
0.3326*15 ==4.989

True

要提取二度联系人聚合特征，就是在一度联系人聚合特征的基础上再做一次聚合即可。

In [388]:
test.update_all(dgl.function.copy_u('aggsum_trans_amount_sum', 'm'),dgl.function.sum('m','2d-aggsum_trans_amount_sum'))

总节点数为26077，有1度联系人的节点有11669个，但是有二度联系人就只有6872个了。

In [397]:
sum(test.ndata['aggsum_trans_amount_sum']>0)

tensor(11669)

In [396]:
sum(test.ndata['2d-aggsum_trans_amount_sum']>0)

tensor(6872)